## The Predicting Prescriber induced Overdose

##### DATA SCIENCE IMMERSIVE FINAL CAPE STONE

This capestone was initaited with an idea that why are so much more physicians presecribing medications that can lead to overdose. I have worked with patients for more than a year in a clinic as well as a hospital and saw significant amount of patients requesting pain relief medications again and again. I have seen people i know been affected with the chronic and life threatening overdose.

The objective of this capestone is to develop models to identify types of prescribers that are a high-risk for opioid related fatalities and predict most influential opioids leading to deaths due to overdose.

The public health data was extracted from [CDC](https://wonder.cdc.gov/) I downloaded into individual tab separated text files as dowloading the 3.2GB data at once and extracting needs significant processing power besides I noticed some anomalies when extracting data from Wonder in large due to the system or functionality of wonder tool they have to group the needed data. The otherData was extraced from CMS Medicare [Part D Opioid Prescriber Data](	https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Part-D-Prescriber.html) and [cms](https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/OpioidMap.html)

**Author : Kiros Gebremariam**

Cohorts of the Data Science Immersive, General Assembly @ Washington DC campus

Loadding Necessary Libraries

In [1]:
import os                                                                    # os library
import numpy as np                                                           # numpy library
import pandas as pd                                                          # pandas library
import matplotlib.pylab as plt                                               # plotting and visualization library
import seaborn as sns                                                        # plotting
import statsmodels.api as sm                                                 # for running regressions
np.random.seed(2018)                                                         # Random seed for replication 



# Configure visual settings:
plt.style.use('fivethirtyeight')
sns.set(font_scale=2)
cmap = sns.diverging_palette(220, 10, as_cmap=True) # one of the many color mappings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10.0, 8.0) 
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
import os                      # just ensuring the files using as working Directory(pwd)
print(os.listdir())

['.DS_Store', '.git', '.gitignore', '.ipynb_checkpoints', '1.1. Capestone Data setup_1.ipynb', 'inputs', 'LICENSE', 'README.md']


In [3]:
df1 = pd.read_csv('./inputs/Medicare_Part_D_Opioid_Prescriber_Summary_File_2016.csv')
df2 =pd.read_csv('./inputs/Physician_Locator.csv')
df2 =pd.read_csv('./inputs/Prescriber_state.csv')
df3 =pd.read_csv('./inputs/Prescriber_county.csv')
df4 =pd.read_csv('./inputs/Prescriber_zip.csv')
df5 =pd.read_csv('./inputs/KFFMentalHealthspendState.csv')
df6 =pd.read_csv('./inputs/pop-urban-pct-historical_cleaned_up.csv')
df7 = pd.read_csv('./inputs/rawdataKFFopioiddeath.csv', engine ="python")
df8 = pd.read_csv('./inputs/Wonder_Drug_Death_Data_1.csv')
df9 = pd.read_csv('./inputs/Wonder_Drug_Death_Data_2.csv')
psr_df= pd.read_csv('./inputs/prescriber_state_rate.csv')
pcr_df = pd.read_csv('./inputs/prescriber_county_rate.csv')
pzr_df= pd.read_csv('./inputs/prescriber_zip_rate.csv')
violentcrime =pd.read_csv('./inputs/violentcrime19992014.csv')
propertycrime =pd.read_csv('./inputs/propertycrime19992014.csv')
df10 = pd.read_csv('./inputs/NHE2016.csv',engine='python')

# Checking Data Integrity

# Checking data Integrity and Quality

In [4]:
df1.head()[:2]  # the data is about the Medicare part D prescriber summary extracted from cms

,NPI,NPPES Provider Last Name,NPPES Provider First Name,NPPES Provider ZIP Code,NPPES Provider State,Specialty Description,Total Claim Count,Opioid Claim Count,Opioid Prescribing Rate,Extended-Release Opioid Claims,Extended-Release Opioid Prescribing Rate
0,1003000126,ENKESHAFI,ARDALAN,21502.0,MD,Internal Medicine,545,23.0,4.22%,NaN,NaN
1,1003000142,KHALIL,RASHID,43623.0,OH,Anesthesiology,1733,1004.0,57.93%,63.0,6.27%


In [5]:
print("--==<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>====")
print((df1.isnull().sum()/df1.shape[0]).sort_values(ascending=False) )  
print("--------------------------------------------------------------------------------------------------------")

--==<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>====
Extended-Release Opioid Prescribing Rate    0.656609
Opioid Prescribing Rate                     0.281917
Opioid Claim Count                          0.281917
Extended-Release Opioid Claims              0.113573
NPPES Provider ZIP Code                     0.000047
NPPES Provider First Name                   0.000028
NPPES Provider Last Name                    0.000028
Total Claim Count                           0.000000
Specialty Description                       0.000000
NPPES Provider State                        0.000000
NPI                                         0.000000
dtype: float64
--------------------------------------------------------------------------------------------------------


In [6]:
list(df1)

['NPI',
 'NPPES Provider Last Name',
 'NPPES Provider First Name',
 'NPPES Provider ZIP Code',
 'NPPES Provider State',
 'Specialty Description',
 'Total Claim Count',
 'Opioid Claim Count',
 'Opioid Prescribing Rate',
 'Extended-Release Opioid Claims',
 'Extended-Release Opioid Prescribing Rate']

In [7]:
#  extracted from from CMS Medicare Part D Opioid Prescribing Geographic 2013-2016 Excel file and 
# created CSV files for the three taps 1) state 2) county and 3) zip
# Stored presciber-state file in a variable(df2)
df2.fillna('National')[:2]   # the data is about the prescriber/Physician_Locator

,State_Name,State_Abbreviation,State_FIPS,2013_Part_ D_Prescribers,2013_Part_Opioid_Prescribers,2013_Opioid Claims,2013_Extended_Release_Opioid_Claims,2013_Overall_Claims,2013_Opioid_Prescribing_Rate,2013_Extended_Release_Opioid_Prescribing_Rate,...,2016_Part D Opioid Prescribers,2016_Opioid Claims,2016_Extended Release Opioid Claims,2016_Overall_Claims,2016_Opioid_Prescribing_Rate,2016_Extended_Release_Opioid_Prescribing_ Rate,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate,2013_2015_Change_in_Opioid_Prescribing_Rate,2013_2015_Change_in_Extended_Release_Opioid_Prescribing_Rate
0,National,National,National,"1,037,770","491,500","78,045,683","4,773,286","1,342,096,654",5.82,6.12,...,496350,77716608,5381275,1452978228,5.34,6.92,-0.47,0.80,-0.30,0.59
1,Alabama,AL,1,"12,820","7,422","2,260,284","118,523","29,160,952",7.75,5.24,...,"7,004","2,156,504","120,579","28,882,148",7.47,5.59,-0.28,0.35,-0.21,0.31


In [8]:
print("--==<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>====")
print((df2.isnull().sum()/df2.shape[0]).sort_values(ascending=False) )  
print("--------------------------------------------------------------------------------------------------------")

--==<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>====
State_Abbreviation                                               0.019231
State_FIPS                                                       0.019231
2013_2015_Change_in_Extended_Release_Opioid_Prescribing_Rate     0.000000
2013_Extended_Release_Opioid_Prescribing_Rate                    0.000000
2014_Overall_Claims                                              0.000000
2014_Extended_Release_Opioid_Claims                              0.000000
2014_Opioid_Claims                                               0.000000
2014_Part D_Opioid_Prescribers                                   0.000000
2014_Part_ D_Prescribers                                         0.000000
2013_Overall_Claims                                              0.000000
2013_Opioid_Prescribing_Rate                                     0.000000
2014_Extended_Release_Opioid_Prescribing_Rate                    0.000000
2013_E

In [9]:
# here just filling the NaN value due to the columns mismatach as there is no need fo data on 
# state abrevation and state state FIPs
df2.fillna("National")[:5] 

,State_Name,State_Abbreviation,State_FIPS,2013_Part_ D_Prescribers,2013_Part_Opioid_Prescribers,2013_Opioid Claims,2013_Extended_Release_Opioid_Claims,2013_Overall_Claims,2013_Opioid_Prescribing_Rate,2013_Extended_Release_Opioid_Prescribing_Rate,...,2016_Part D Opioid Prescribers,2016_Opioid Claims,2016_Extended Release Opioid Claims,2016_Overall_Claims,2016_Opioid_Prescribing_Rate,2016_Extended_Release_Opioid_Prescribing_ Rate,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate,2013_2015_Change_in_Opioid_Prescribing_Rate,2013_2015_Change_in_Extended_Release_Opioid_Prescribing_Rate
0,National,National,National,"1,037,770","491,500","78,045,683","4,773,286","1,342,096,654",5.82,6.12,...,496350,77716608,5381275,1452978228,5.34,6.92,-0.47,0.80,-0.30,0.59
1,Alabama,AL,1,"12,820","7,422","2,260,284","118,523","29,160,952",7.75,5.24,...,"7,004","2,156,504","120,579","28,882,148",7.47,5.59,-0.28,0.35,-0.21,0.31
2,Alaska,AK,2,"2,275","1,099","86,517","8,602","1,281,057",6.75,9.94,...,"1,043","82,557","9,074","1,225,327",6.74,10.99,-0.01,1.05,-0.09,0.71
3,Arizona,AZ,4,"20,542","10,510","1,545,138","151,086","22,126,421",6.98,9.78,...,"11,026","1,621,223","173,171","24,891,001",6.51,10.68,-0.47,0.90,-0.20,0.48
4,Arkansas,AR,5,"7,909","4,700","1,128,356","57,519","16,759,116",6.73,5.10,...,"4,854","1,155,171","65,819","18,139,502",6.37,5.70,-0.36,0.60,-0.36,0.49


In [10]:
list(df2)

['State_Name',
 'State_Abbreviation',
 'State_FIPS',
 '2013_Part_ D_Prescribers',
 '2013_Part_Opioid_Prescribers',
 '2013_Opioid Claims',
 '2013_Extended_Release_Opioid_Claims',
 '2013_Overall_Claims',
 '2013_Opioid_Prescribing_Rate',
 '2013_Extended_Release_Opioid_Prescribing_Rate',
 '2014_Part_ D_Prescribers',
 '2014_Part D_Opioid_Prescribers',
 '2014_Opioid_Claims',
 '2014_Extended_Release_Opioid_Claims',
 '2014_Overall_Claims',
 '2014_Opioid_Prescribing_Rate',
 '2014_Extended_Release_Opioid_Prescribing_Rate',
 '2015_Part D Prescribers',
 '2015_Part D Opioid Prescribers',
 '2015_Opioid Claims',
 '2015_Extended Release Opioid Claims',
 '2015_Overall_Claims',
 '2015_Opioid_Prescribing_Rate',
 '2015_Extended_Release_Opioid_Prescribing_ Rate',
 '2016_Part D Prescribers',
 '2016_Part D Opioid Prescribers',
 '2016_Opioid Claims',
 '2016_Extended Release Opioid Claims',
 '2016_Overall_Claims',
 '2016_Opioid_Prescribing_Rate',
 '2016_Extended_Release_Opioid_Prescribing_ Rate',
 '2013_2016_C

In [11]:
#Selected the opioid prescribe rate columsn for years 2013-2016 by prescriber state(psr)
# this was extracted from df2 so that it will be important to see for features in the next stages
psr_df.head()[:2]
psr_df.fillna("National")[:2]

,State_Name,State_Abbreviation,2013_Opioid_Prescribing_Rate,2013_Extended_Release_Opioid_Prescribing_Rate,2014_Opioid_Prescribing_Rate,2014_Extended_Release_Opioid_Prescribing_Rate,2015_Opioid_Prescribing_Rate,2015_Extended_Release_Opioid_Prescribing_ Rate,2016_Opioid_Prescribing_Rate,2016_Extended_Release_Opioid_Prescribing_ Rate,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate
0,National,National,5.82,6.12,5.74,6.27,5.52,6.71,5.34,6.92,-0.47,0.80
1,Alabama,AL,7.75,5.24,7.86,5.24,7.54,5.55,7.47,5.59,-0.28,0.35


In [12]:

print("--==<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>========")
print((psr_df.isnull().sum()/psr_df.shape[0]).sort_values(ascending=False))  
print("-------------------------------------------------------------------------------------------------------------")

--==<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>========
State_Abbreviation                                               0.019231
2013_2016_Change in Extended Release Opioid Prescribing Rate     0.000000
2013_2016_Change in Opioid Prescribing Rate                      0.000000
2016_Extended_Release_Opioid_Prescribing_ Rate                   0.000000
2016_Opioid_Prescribing_Rate                                     0.000000
2015_Extended_Release_Opioid_Prescribing_ Rate                   0.000000
2015_Opioid_Prescribing_Rate                                     0.000000
2014_Extended_Release_Opioid_Prescribing_Rate                    0.000000
2014_Opioid_Prescribing_Rate                                     0.000000
2013_Extended_Release_Opioid_Prescribing_Rate                    0.000000
2013_Opioid_Prescribing_Rate                                     0.000000
State_Name                                                       0.000000
dt

In [13]:
psr_df.fillna("National")
psr_df.head()[:2]

,State_Name,State_Abbreviation,2013_Opioid_Prescribing_Rate,2013_Extended_Release_Opioid_Prescribing_Rate,2014_Opioid_Prescribing_Rate,2014_Extended_Release_Opioid_Prescribing_Rate,2015_Opioid_Prescribing_Rate,2015_Extended_Release_Opioid_Prescribing_ Rate,2016_Opioid_Prescribing_Rate,2016_Extended_Release_Opioid_Prescribing_ Rate,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate
0,National,NaN,5.82,6.12,5.74,6.27,5.52,6.71,5.34,6.92,-0.47,0.80
1,Alabama,AL,7.75,5.24,7.86,5.24,7.54,5.55,7.47,5.59,-0.28,0.35


In [14]:
psr_changeex_df = psr_df[['State_Name','2013_2016_Change in Opioid Prescribing Rate', '2013_2016_Change in Extended Release Opioid Prescribing Rate ']]
psr_changeex_df .head()[:2]

,State_Name,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate
0,National,-0.47,0.80
1,Alabama,-0.28,0.35


In [15]:
#Selected the opioid prescribe rate columsn for years 2013-2016 by prescriber county
# pcr_df.head()[:2]
psr_df.fillna("National")[:2]

,State_Name,State_Abbreviation,2013_Opioid_Prescribing_Rate,2013_Extended_Release_Opioid_Prescribing_Rate,2014_Opioid_Prescribing_Rate,2014_Extended_Release_Opioid_Prescribing_Rate,2015_Opioid_Prescribing_Rate,2015_Extended_Release_Opioid_Prescribing_ Rate,2016_Opioid_Prescribing_Rate,2016_Extended_Release_Opioid_Prescribing_ Rate,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate
0,National,National,5.82,6.12,5.74,6.27,5.52,6.71,5.34,6.92,-0.47,0.80
1,Alabama,AL,7.75,5.24,7.86,5.24,7.54,5.55,7.47,5.59,-0.28,0.35


In [16]:
print("--==<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>=========")
print((pcr_df.isnull().sum()/pcr_df.shape[0]).sort_values(ascending=False)  )  
print("--------------------------------------------------------------------------------------------------------------")

--==<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>=========
2013_2016_Change in Extended Release Opioid Prescribing Rate     0.005509
2013_2016_Change in Opioid Prescribing Rate                      0.005509
2015_Extended_Release_Opioid_Prescribing_ Rate                   0.002592
2015_Opioid_Prescribing_Rate                                     0.002592
2016_Extended_Release_Opioid_Prescribing_ Rate                   0.002268
2016_Opioid_Prescribing_Rate                                     0.002268
2014_Extended_Release_Opioid_Prescribing_Rate                    0.001944
2014_Opioid_Prescribing_Rate                                     0.001944
2013_Extended_Release_Opioid_Prescribing_Rate                    0.001944
2013_Opioid_Prescribing_Rate                                     0.001944
State_FIPS                                                       0.000324
County Name                                                      0.000324
S

In [17]:
pcr_df.fillna("National")[:2]
# pcr_df.head()[:2]

,State_Name,State_Abbreviation,County Name,State_FIPS,2013_Opioid_Prescribing_Rate,2013_Extended_Release_Opioid_Prescribing_Rate,2014_Opioid_Prescribing_Rate,2014_Extended_Release_Opioid_Prescribing_Rate,2015_Opioid_Prescribing_Rate,2015_Extended_Release_Opioid_Prescribing_ Rate,2016_Opioid_Prescribing_Rate,2016_Extended_Release_Opioid_Prescribing_ Rate,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate
0,National,National,National,National,5.82,6.12,5.74,6.27,5.52,6.71,5.34,6.92,-0.47,0.8
1,Alabama,AL,Autauga,1001,9.8,4.78,9.72,5.85,8.07,7.24,11.47,9.39,1.67,4.61


In [18]:
list(pcr_df)

['State_Name',
 'State_Abbreviation',
 'County Name',
 'State_FIPS',
 '2013_Opioid_Prescribing_Rate',
 '2013_Extended_Release_Opioid_Prescribing_Rate',
 '2014_Opioid_Prescribing_Rate',
 '2014_Extended_Release_Opioid_Prescribing_Rate',
 '2015_Opioid_Prescribing_Rate',
 '2015_Extended_Release_Opioid_Prescribing_ Rate',
 '2016_Opioid_Prescribing_Rate',
 '2016_Extended_Release_Opioid_Prescribing_ Rate',
 '2013_2016_Change in Opioid Prescribing Rate',
 '2013_2016_Change in Extended Release Opioid Prescribing Rate ']

In [19]:
pcr_changeex_df = pcr_df[['State_Name','2013_2016_Change in Opioid Prescribing Rate', '2013_2016_Change in Extended Release Opioid Prescribing Rate ']]
pcr_changeex_df .head()[:2]

,State_Name,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate
0,National,-0.47,0.80
1,Alabama,1.67,4.61


In [20]:
print("--==<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>=========")
print((df2.isnull().sum()/df2.shape[0]).sort_values(ascending=False)   )  
print("--------------------------------------------------------------------------------------------------------------")

--==<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>=========
State_Abbreviation                                               0.019231
State_FIPS                                                       0.019231
2013_2015_Change_in_Extended_Release_Opioid_Prescribing_Rate     0.000000
2013_Extended_Release_Opioid_Prescribing_Rate                    0.000000
2014_Overall_Claims                                              0.000000
2014_Extended_Release_Opioid_Claims                              0.000000
2014_Opioid_Claims                                               0.000000
2014_Part D_Opioid_Prescribers                                   0.000000
2014_Part_ D_Prescribers                                         0.000000
2013_Overall_Claims                                              0.000000
2013_Opioid_Prescribing_Rate                                     0.000000
2014_Extended_Release_Opioid_Prescribing_Rate                    0.000000
2

In [21]:
list(df3)

['State_Name',
 'State_Abbreviation',
 'County Name',
 'State_FIPS',
 '2013_Part_ D_Prescribers',
 '2013_Part_Opioid_Prescribers',
 '2013_Opioid Claims',
 '2013_Extended_Release_Opioid_Claims',
 '2013_Overall_Claims',
 '2013_Opioid_Prescribing_Rate',
 '2013_Extended_Release_Opioid_Prescribing_Rate',
 '2014_Part_ D_Prescribers',
 '2014_Part D_Opioid_Prescribers',
 '2014_Opioid_Claims',
 '2014_Extended_Release_Opioid_Claims',
 '2014_Overall_Claims',
 '2014_Opioid_Prescribing_Rate',
 '2014_Extended_Release_Opioid_Prescribing_Rate',
 '2015_Part D Prescribers',
 '2015_Part D Opioid Prescribers',
 '2015_Opioid Claims',
 '2015_Extended Release Opioid Claims',
 '2015_Overall_Claims',
 '2015_Opioid_Prescribing_Rate',
 '2015_Extended_Release_Opioid_Prescribing_ Rate',
 '2013_2015_Change_in_Opioid_Prescribing_Rate',
 '2013_2015_Change_in_Extended_Release_Opioid_Prescribing_Rate ']

In [22]:
df3.head()[:2]

,State_Name,State_Abbreviation,County Name,State_FIPS,2013_Part_ D_Prescribers,2013_Part_Opioid_Prescribers,2013_Opioid Claims,2013_Extended_Release_Opioid_Claims,2013_Overall_Claims,2013_Opioid_Prescribing_Rate,...,2014_Extended_Release_Opioid_Prescribing_Rate,2015_Part D Prescribers,2015_Part D Opioid Prescribers,2015_Opioid Claims,2015_Extended Release Opioid Claims,2015_Overall_Claims,2015_Opioid_Prescribing_Rate,2015_Extended_Release_Opioid_Prescribing_ Rate,2013_2015_Change_in_Opioid_Prescribing_Rate,2013_2015_Change_in_Extended_Release_Opioid_Prescribing_Rate
0,National,NaN,NaN,NaN,"1,037,770","491,500","78,045,683","4,773,286","1,342,096,654",5.82,...,6.27,"1,090,979","496,383","78,372,855","5,257,481","1,418,618,875",5.52,6.71,-0.30,0.59
1,Alabama,AL,Autauga,1001.0,67,44,"20,482",979,"208,936",9.80,...,5.85,68,43,"15,998","1,159","198,347",8.07,7.24,-1.73,2.46


In [23]:
print("--====<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>=======")
print((df3.isnull().sum()/df3.shape[0]).sort_values(ascending=False))  
print("--------------------------------------------------------------------------------------------------------------")

--====<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>=======
2013_2015_Change_in_Extended_Release_Opioid_Prescribing_Rate     0.004537
2013_2015_Change_in_Opioid_Prescribing_Rate                      0.004537
2015_Extended_Release_Opioid_Prescribing_ Rate                   0.002592
2015_Opioid_Prescribing_Rate                                     0.002592
2015_Overall_Claims                                              0.002592
2015_Extended Release Opioid Claims                              0.002592
2015_Opioid Claims                                               0.002592
2015_Part D Opioid Prescribers                                   0.002592
2015_Part D Prescribers                                          0.002592
2013_Extended_Release_Opioid_Prescribing_Rate                    0.001944
2013_Part_ D_Prescribers                                         0.001944
2013_Part_Opioid_Prescribers                                     0.001944
2

In [24]:
list(df3)   # this list is for prescriber county information based on cms datasets

['State_Name',
 'State_Abbreviation',
 'County Name',
 'State_FIPS',
 '2013_Part_ D_Prescribers',
 '2013_Part_Opioid_Prescribers',
 '2013_Opioid Claims',
 '2013_Extended_Release_Opioid_Claims',
 '2013_Overall_Claims',
 '2013_Opioid_Prescribing_Rate',
 '2013_Extended_Release_Opioid_Prescribing_Rate',
 '2014_Part_ D_Prescribers',
 '2014_Part D_Opioid_Prescribers',
 '2014_Opioid_Claims',
 '2014_Extended_Release_Opioid_Claims',
 '2014_Overall_Claims',
 '2014_Opioid_Prescribing_Rate',
 '2014_Extended_Release_Opioid_Prescribing_Rate',
 '2015_Part D Prescribers',
 '2015_Part D Opioid Prescribers',
 '2015_Opioid Claims',
 '2015_Extended Release Opioid Claims',
 '2015_Overall_Claims',
 '2015_Opioid_Prescribing_Rate',
 '2015_Extended_Release_Opioid_Prescribing_ Rate',
 '2013_2015_Change_in_Opioid_Prescribing_Rate',
 '2013_2015_Change_in_Extended_Release_Opioid_Prescribing_Rate ']

In [25]:
df4.head()[:2] # this is the dataset for the prescriber zip which is one of the dataclassifications by cms

,State_Name,State_Abbreviation,State_FIPS,2013_Part_ D_Prescribers,2013_Part_Opioid_Prescribers,2013_Opioid Claims,2013_Extended_Release_Opioid_Claims,2013_Overall_Claims,2013_Opioid_Prescribing_Rate,2013_Extended_Release_Opioid_Prescribing_Rate,...,2014_Extended_Release_Opioid_Prescribing_Rate,2015_Part D Prescribers,2015_Part D Opioid Prescribers,2015_Opioid Claims,2015_Extended Release Opioid Claims,2015_Overall_Claims,2015_Opioid_Prescribing_Rate,2015_Extended_Release_Opioid_Prescribing_ Rate,2013_2015_Change_in_Opioid_Prescribing_Rate,2013_2015_Change_in_Extended_Release_Opioid_Prescribing_Rate
0,National,NaN,NaN,"1,037,770","491,500","78,045,683","4,773,286","1,342,096,654",5.82,6.12,...,6.27,"1,090,979","496,383","78,372,855","5,257,481","1,418,618,875",5.52,6.71,-0.30,0.59
1,Alabama,AL,35004.0,18,9,"1,340",19,"18,632",7.19,1.42,...,6.30,17,7,"1,340",39,"16,830",7.96,2.91,0.77,1.49


In [26]:
#Selected the opioid prescribe rate columsn for years 2013-2016 by prescriber county_rate
# pcr_df.head()[:2]
pcr_df.fillna("National")[:2]

,State_Name,State_Abbreviation,County Name,State_FIPS,2013_Opioid_Prescribing_Rate,2013_Extended_Release_Opioid_Prescribing_Rate,2014_Opioid_Prescribing_Rate,2014_Extended_Release_Opioid_Prescribing_Rate,2015_Opioid_Prescribing_Rate,2015_Extended_Release_Opioid_Prescribing_ Rate,2016_Opioid_Prescribing_Rate,2016_Extended_Release_Opioid_Prescribing_ Rate,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate
0,National,National,National,National,5.82,6.12,5.74,6.27,5.52,6.71,5.34,6.92,-0.47,0.8
1,Alabama,AL,Autauga,1001,9.8,4.78,9.72,5.85,8.07,7.24,11.47,9.39,1.67,4.61


In [27]:
pcr_changeex_df = pcr_df[['State_Name','2013_2016_Change in Opioid Prescribing Rate', '2013_2016_Change in Extended Release Opioid Prescribing Rate ']]
pcr_changeex_df .head()[:2]

,State_Name,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate
0,National,-0.47,0.80
1,Alabama,1.67,4.61


In [28]:
print("--====<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>=======")
print((pzr_df.isnull().sum()/pzr_df.shape[0]).sort_values(ascending=False) )  
print("--------------------------------------------------------------------------------------------------------------")

--====<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>=======
2013_2016_Change in Extended Release Opioid Prescribing Rate     0.060631
2013_2016_Change in Opioid Prescribing Rate                      0.060631
2015_Extended_Release_Opioid_Prescribing_ Rate                   0.029482
2015_Opioid_Prescribing_Rate                                     0.029482
2016_Extended Release Opioid Prescribing Rate                    0.029013
2016_Opioid Prescribing Rate                                     0.029013
2014_Extended_Release_Opioid_Prescribing_Rate                    0.028597
2014_Opioid_Prescribing_Rate                                     0.028597
2013_Extended_Release_Opioid_Prescribing_Rate                    0.027659
2013_Opioid_Prescribing_Rate                                     0.027659
State_FIPS                                                       0.000052
State_Abbreviation                                               0.000052
S

In [29]:
pzr_df.fillna("National")[:2]
# pzr_df.head()[:2] here the data is by the prescriber zib which is prescriber zip rate
# flling the NaN values under the stateabbrevation and stateFips with National

,State_Name,State_Abbreviation,State_FIPS,2013_Opioid_Prescribing_Rate,2013_Extended_Release_Opioid_Prescribing_Rate,2014_Opioid_Prescribing_Rate,2014_Extended_Release_Opioid_Prescribing_Rate,2015_Opioid_Prescribing_Rate,2015_Extended_Release_Opioid_Prescribing_ Rate,2016_Opioid Prescribing Rate,2016_Extended Release Opioid Prescribing Rate,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate
0,National,National,National,5.82,6.12,5.74,6.27,5.52,6.71,5.34,6.92,-0.47,0.8
1,Alabama,AL,35004,7.19,1.42,9.71,6.3,7.96,2.91,7.47,5.61,0.28,4.19


In [30]:
list(pzr_df)

['State_Name',
 'State_Abbreviation',
 'State_FIPS',
 '2013_Opioid_Prescribing_Rate',
 '2013_Extended_Release_Opioid_Prescribing_Rate',
 '2014_Opioid_Prescribing_Rate',
 '2014_Extended_Release_Opioid_Prescribing_Rate',
 '2015_Opioid_Prescribing_Rate',
 '2015_Extended_Release_Opioid_Prescribing_ Rate',
 '2016_Opioid Prescribing Rate',
 '2016_Extended Release Opioid Prescribing Rate',
 '2013_2016_Change in Opioid Prescribing Rate',
 '2013_2016_Change in Extended Release Opioid Prescribing Rate ']

In [31]:
pzr_changeex_df = pzr_df[['State_Name','2013_2016_Change in Opioid Prescribing Rate', '2013_2016_Change in Extended Release Opioid Prescribing Rate ']]
pzr_changeex_df .head()[:2]

,State_Name,2013_2016_Change in Opioid Prescribing Rate,2013_2016_Change in Extended Release Opioid Prescribing Rate
0,National,-0.47,0.80
1,Alabama,0.28,4.19


In [32]:
df5.head()[:2]  # 

,Location,FY2004__SMHA Expenditures Per Capita,FY2005__SMHA Expenditures Per Capita,FY2006__SMHA Expenditures Per Capita,FY2007__SMHA Expenditures Per Capita,FY2008__SMHA Expenditures Per Capita,FY2009__SMHA Expenditures Per Capita,FY2010__SMHA Expenditures Per Capita,FY2011__SMHA Expenditures Per Capita,FY2012__SMHA Expenditures Per Capita,FY2013__SMHA Expenditures Per Capita,Footnotes
0,United States,93.04,99.55,103.53,113.27,121.13,122.90,120.56,123.93,124.99,119.62,NaN
1,Alabama,58.78,60.31,64.43,74.03,79.39,77.89,78.19,78.33,76.27,72.64,NaN


In [33]:
print("--====<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>=======")
print((df5.isnull().sum()/df5.shape[0]).sort_values(ascending=False))  
print("--------------------------------------------------------------------------------------------------------------")

--====<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>=======
Footnotes                               0.584416
FY2007__SMHA Expenditures Per Capita    0.337662
FY2010__SMHA Expenditures Per Capita    0.324675
FY2009__SMHA Expenditures Per Capita    0.324675
FY2008__SMHA Expenditures Per Capita    0.324675
FY2006__SMHA Expenditures Per Capita    0.324675
FY2005__SMHA Expenditures Per Capita    0.324675
FY2004__SMHA Expenditures Per Capita    0.324675
FY2013__SMHA Expenditures Per Capita    0.311688
FY2012__SMHA Expenditures Per Capita    0.311688
FY2011__SMHA Expenditures Per Capita    0.311688
Location                                0.103896
dtype: float64
--------------------------------------------------------------------------------------------------------------


In [34]:
list(df5)

['Location',
 'FY2004__SMHA Expenditures Per Capita',
 'FY2005__SMHA Expenditures Per Capita',
 'FY2006__SMHA Expenditures Per Capita',
 'FY2007__SMHA Expenditures Per Capita',
 'FY2008__SMHA Expenditures Per Capita',
 'FY2009__SMHA Expenditures Per Capita',
 'FY2010__SMHA Expenditures Per Capita',
 'FY2011__SMHA Expenditures Per Capita',
 'FY2012__SMHA Expenditures Per Capita',
 'FY2013__SMHA Expenditures Per Capita',
 'Footnotes']

In [35]:
df6.head()[:2]

,FIPS,Area Name,1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2010
0,0,United States,39.6,45.6,51.2,56.1,56.5,64.0,69.9,73.6,73.7,75.2,79.0,80.7
1,1,Alabama,11.9,17.3,21.7,28.1,30.2,43.8,54.8,58.6,60.0,60.4,55.4,59.0


In [36]:
list(df6)

['FIPS',
 'Area Name',
 '1900',
 '1910',
 '1920',
 '1930',
 '1940',
 '1950',
 '1960',
 '1970',
 '1980',
 '1990',
 '2000',
 '2010']

In [37]:
print("--====<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>=======")
print((df6.isnull().sum()/df6.shape[0]).sort_values(ascending=False) )  
print("--------------------------------------------------------------------------------------------------------------")

--====<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>=======
2010         0.118644
2000         0.118644
1990         0.118644
1980         0.118644
1970         0.118644
1960         0.118644
1950         0.118644
1940         0.118644
1930         0.118644
1920         0.118644
1910         0.118644
1900         0.118644
Area Name    0.118644
FIPS         0.033898
dtype: float64
--------------------------------------------------------------------------------------------------------------


In [38]:
list(df6)

['FIPS',
 'Area Name',
 '1900',
 '1910',
 '1920',
 '1930',
 '1940',
 '1950',
 '1960',
 '1970',
 '1980',
 '1990',
 '2000',
 '2010']

#### Extra: Mental health and opioids: is the opioid epidemic really a mental health crisis?

This was added to see if there is any relationship  of any kind with the mental health  of the person and opiods as well as to see if its connected with any of the crimes as the US is becoming a hub of gun related deaths across the nation,  in different locations like what has  happened recently the Capital Gazette shooting in Annapolis, MD.
A mass shooting occurred on June 28, 2018, at the offices of newspaper publisher Capital Gazette Communications in Annapolis. The perpetrator killed five Capital Gazette employees and gravely wounded several in the attack on the newsroom[Capital Gazette ](https://www.cnn.com/us/live-news/maryland-newspaper-shooting/index.html)

In [39]:
df8.head()[:2]  # looking on the first two rows of the Dataframe

,Notes,State,State Code,Year,Year Code,Gender,Gender Code,Deaths,Population,Crude Rate
0,NaN,Alabama,1.0,1999.0,1999.0,Female,F,84.0,2293259.0,3.7
1,NaN,Alabama,1.0,1999.0,1999.0,Male,M,111.0,2136882.0,5.2


In [40]:
df8.head()[:2]

,Notes,State,State Code,Year,Year Code,Gender,Gender Code,Deaths,Population,Crude Rate
0,NaN,Alabama,1.0,1999.0,1999.0,Female,F,84.0,2293259.0,3.7
1,NaN,Alabama,1.0,1999.0,1999.0,Male,M,111.0,2136882.0,5.2


In [41]:
df8.columns

Index(['Notes', 'State', 'State Code', 'Year', 'Year Code', 'Gender',
       'Gender Code', 'Deaths', 'Population', 'Crude Rate'],
      dtype='object')

In [42]:
print("--==<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>====")
print((df8.isnull().sum()/df8.shape[0]).sort_values(ascending=False))  
print("-----------------------------------------------------------------------------------------------------------")

--==<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>====
Notes          0.961905
Crude Rate     0.063305
Population     0.038095
Deaths         0.038095
Gender Code    0.038095
Gender         0.038095
Year Code      0.038095
Year           0.038095
State Code     0.038095
State          0.038095
dtype: float64
-----------------------------------------------------------------------------------------------------------


In [43]:
df8.drop(['Notes'], axis=1, inplace=True)                                 # drops Notes column
df8.dropna(axis=0, how='any', inplace=True)                             # drops rows with NaN
df8['Year'] = df8['Year'].astype(int)                              # casts datatypes for columns
df8['Year Code'] = df8['Year Code'].astype(int)
df8['Year Code'] = df8['Year Code'].astype(str)
df8['Deaths'] = df8['Deaths'].astype(int)
df8['Population'] = df8['Population'].astype(int)

# Calculates death rate per 100,000 of population (Deaths / Population) * 100,000
df8['Death Rate'] = df8['Deaths'] / df8['Population'] * 100000
df8.drop('Crude Rate', axis=1, inplace=True)                            # drops Crude Rate column
df8.set_index('Year', inplace=True)   

In [44]:
df8.head()[:2]

,State,State Code,Year Code,Gender,Gender Code,Deaths,Population,Death Rate
Year,,,,,,,,
1999,Alabama,1.0,1999,Female,F,84,2293259,3.662909
1999,Alabama,1.0,1999,Male,M,111,2136882,5.194484


In [45]:
df9.head()[:2]

,Notes,Year,Year Code,State,State Code,Gender,Gender Code,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Crude Rate Standard Error,% of Total Deaths
0,Total,1999.0,1999.0,Alabama,1.0,Female,F,NaN,NaN,21.0,2293259.0,0.9,0.2,0.0
1,NaN,1999.0,1999.0,Alabama,1.0,Male,M,25-34 years,25-34,11.0,300073.0,Unreliable,1.1,0.0


In [46]:
print("--==<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>====")
print((df9.isnull().sum()/df9.shape[0]).sort_values(ascending=False))  
print("-----------------------------------------------------------------------------------------------------------")

--==<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>====
Notes                        0.694154
Ten-Year Age Groups Code     0.305846
Ten-Year Age Groups          0.305846
Gender Code                  0.111634
Gender                       0.111634
State Code                   0.011175
State                        0.011175
Year Code                    0.009175
Year                         0.009175
% of Total Deaths            0.009058
Crude Rate Standard Error    0.009058
Crude Rate                   0.009058
Population                   0.009058
Deaths                       0.009058
dtype: float64
-----------------------------------------------------------------------------------------------------------


In [47]:
df9.columns

Index(['Notes', 'Year', 'Year Code', 'State', 'State Code', 'Gender',
       'Gender Code', 'Ten-Year Age Groups', 'Ten-Year Age Groups Code',
       'Deaths', 'Population', 'Crude Rate', 'Crude Rate Standard Error',
       '% of Total Deaths'],
      dtype='object')

In [48]:
# df9.drop(['Notes'], axis=1, inplace=True)                                 # drops Notes column
df9.dropna(axis=0, how='any', inplace=True)                             # drops rows with NaN
df9['Year'] = df9['Year'].astype(int)                              # casts datatypes for columns
df9['Year Code'] = df9['Year Code'].astype(int)
df9['Year Code'] = df9['Year Code'].astype(str)
df9['Deaths'] = df9['Deaths'].astype(int)
df9['Population'] = df9['Population'].astype(int)

# Calculates death rate per 100,000 of population (Deaths / Population) * 100,000
df9['Death Rate'] = df9['Deaths'] / df9['Population'] * 100000
df9.drop('Crude Rate', axis=1, inplace=True)                            # drops Crude Rate column
df9.set_index('Year', inplace=True)

In [49]:
df9.head()[:2]

,Notes,Year Code,State,State Code,Gender,Gender Code,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate Standard Error,% of Total Deaths,Death Rate
Year,,,,,,,,,,,,,


In [50]:
df9.columns

Index(['Notes', 'Year Code', 'State', 'State Code', 'Gender', 'Gender Code',
       'Ten-Year Age Groups', 'Ten-Year Age Groups Code', 'Deaths',
       'Population', 'Crude Rate Standard Error', '% of Total Deaths',
       'Death Rate'],
      dtype='object')

In [51]:
df10.head()[:2]   # USD in millions

,Expenditure Amount (Millions),1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Total National Health Expenditures,"27,214","29,138","31,842","34,595","38,394","41,852","46,081","51,565","58,402",...,"2,295,307","2,399,121","2,495,414","2,598,823","2,689,349","2,797,260","2,879,008","3,026,157","3,200,815","3,337,248"
1,Out of pocket,"12,949","13,357","14,255","15,311","16,928","18,209","18,594","18,537","20,500",...,"290,000","295,184","293,723","299,742","309,973","318,330","325,189","330,068","339,315","352,537"


In [52]:
print("--==<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>====")
print((df10.isnull().sum()/df10.shape[0]).sort_values(ascending=False))  
print("-----------------------------------------------------------------------------------------------------------")

--==<checking the null values of the dataframe and sorting them as percentage of the dataframe shape>====
1967                             0.090239
1960                             0.090239
1961                             0.090239
1962                             0.090239
1963                             0.090239
1964                             0.090239
1965                             0.090239
1966                             0.090239
1968                             0.090239
1969                             0.090239
1983                             0.086556
1980                             0.086556
1981                             0.086556
1982                             0.086556
2016                             0.086556
1984                             0.086556
1985                             0.086556
1978                             0.086556
1979                             0.086556
1972                             0.086556
1977                             0.086556
1976        

In [53]:
list(df10)

['Expenditure Amount (Millions)',
 '1960',
 '1961',
 '1962',
 '1963',
 '1964',
 '1965',
 '1966',
 '1967',
 '1968',
 '1969',
 '1970',
 '1971',
 '1972',
 '1973',
 '1974',
 '1975',
 '1976',
 '1977',
 '1978',
 '1979',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016']

In [54]:
# extract Total NHE and Total CMS Program rows
df10 = df10.loc[[0,35,36],]
df10 = df10.set_index("Expenditure Amount (Millions)")

# set to numeric for plotting purposes
df10 = df10.apply(pd.to_numeric, errors='ignore')


df10.head()[:2]

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
Expenditure Amount (Millions),,,,,,,,,,,,,,,,,,,,,
Total National Health Expenditures,"27,214","29,138","31,842","34,595","38,394","41,852","46,081","51,565","58,402","65,923",...,"2,295,307","2,399,121","2,495,414","2,598,823","2,689,349","2,797,260","2,879,008","3,026,157","3,200,815","3,337,248"
"Total CMS Programs (Medicaid, CHIP and Medicare)",-,-,-,-,-,-,"3,146","8,065","9,759","11,220",...,"767,613","821,394","884,386","928,553","963,412","1,004,951","1,049,032","1,128,749","1,207,734","1,254,525"


In [55]:
# Extract series of spending data 1999 - 2016 to compare against opioid deaths
cms_spending = df10.loc['Total CMS Programs (Medicaid, CHIP and Medicare)','1999':'2016']
nhe_spending = df10.loc['Total National Health Expenditures','1999':'2016']
# percentage = cms_spending / nhe_spending

# percentage.max()

In [56]:
cms_spending.head()[:2]

1999    398,350
2000    428,187
Name: Total CMS Programs (Medicaid, CHIP and Medicare), dtype: object

In [57]:
nhe_spending.head()[:2]

1999    1,277,700
2000    1,369,125
Name: Total National Health Expenditures, dtype: object

In [58]:
nhe_spending

1999    1,277,700
2000    1,369,125
2001    1,486,158
2002    1,628,624
2003    1,767,567
2004    1,895,724
2005    2,023,744
2006    2,156,158
2007    2,295,307
2008    2,399,121
2009    2,495,414
2010    2,598,823
2011    2,689,349
2012    2,797,260
2013    2,879,008
2014    3,026,157
2015    3,200,815
2016    3,337,248
Name: Total National Health Expenditures, dtype: object